# Data transformation pipelines

Several steps:
* clean data from missing values
* encode categorical features
* rescale numerical features

### Imports

In [1]:
import os
import numpy as np
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# better visualization for long outputs
#from IPython.core.display import HTML
#display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Data

In [3]:
data_path = '/home/lorenzo/skl-repo/0_data/california_housing.csv'
df = pd.read_csv(data_path)

In [4]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state=3542)
print(f'Train set length: {len(train)}')
print(f'Test set length: {len(test)}')

Train set length: 16512
Test set length: 4128


### Managing null values

In [6]:
print(f'df original shape: {df.shape}')
df.isna().sum()

df original shape: (20640, 10)


longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Option 1: remove rows containing null values.

In [7]:
df.dropna().shape

(20433, 10)

Option 2: remove columns containing null values.

In [8]:
df.drop('total_bedrooms', axis=1).shape

(20640, 9)

Option 3: fill null values with mean, median or other value.

In [9]:
median = df['total_bedrooms'].median()
df['total_bedrooms'].fillna(median, inplace=True)
df.shape

(20640, 10)

Notice however, that it would be prefereable to compute the median (or mean, or other) only on the training set and then use it to fill the missing values in both the training and test set.

In [10]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')

We need to momentarily separate numerical features (which can be filled with mean/median) from categorical/string features which can be filled with constant or most frequent values. In this case, we'll simply create a copy of the dataframe without the column 'ocean_proximity'.

In [11]:
train_num = train.drop('ocean_proximity', axis=1)
test_num = test.drop('ocean_proximity', axis=1)

Now we can fit and transform the training set.

In [12]:
train_num_cl = imputer.fit_transform(train_num)

Then we apply the same transformation to the test set, without refitting it.

In [13]:
test_num_cl = imputer.transform(test_num)

Put everything back together.

In [14]:
train_clean = pd.DataFrame(train_num_cl, columns = train_num.columns, index = train_num.index)
train_clean['ocean_proximity'] = train['ocean_proximity']
test_clean = pd.DataFrame(test_num_cl, columns = test_num.columns, index = test_num.index)
test_clean['ocean_proximity'] = test['ocean_proximity']

In [15]:
print(f'Train set length: {len(train)}')
print(f'Test set length: {len(test)}')

Train set length: 16512
Test set length: 4128


In [16]:
train_clean.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Now, it is a good idea to separate categorical from numerical features, as their paths temporarily diverge.

In [17]:
train_cat = train_clean.select_dtypes(exclude=np.number)
train_num = train_clean.select_dtypes(include=np.number)
test_cat = test_clean.select_dtypes(exclude=np.number)
test_num = test_clean.select_dtypes(include=np.number)

In [18]:
print(f'Numerical columns: {train_num.columns}')
print(f'Categorical columns: {train_cat.columns}')

Numerical columns: Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')
Categorical columns: Index(['ocean_proximity'], dtype='object')


### Encoding categorical features
Binary features like male/female, can be simply encoded by using one dummy variable. This is equivalent to applying one-hot-encoding.

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# ...

When dealing with multi-class categorical features one possible approach is **ordinal encoding**, meaning that a numerical value is attached to each possible original category. However, these numerical labels will have an ordinal dimension, meaning that features whose numerical label is closer, will be interpreted as being more similar.

In [20]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
train_cat_enc = ord_enc.fit_transform(train_cat)
test_cat_enc = ord_enc.transform(test_cat)
print(train_cat.head(5))
print(f'\nAfter encoding: \n{train_cat_enc[:5]}')

      ocean_proximity
4781        <1H OCEAN
1512         NEAR BAY
14137      NEAR OCEAN
2912           INLAND
2204           INLAND

After encoding: 
[[0.]
 [3.]
 [4.]
 [1.]
 [1.]]


This issue can be fixed by creating a binary attribute (i.e. a dummy variable) for each category (minus one, if the model has a constant). This is called **one-hot-encoding**.

In [21]:
from sklearn.preprocessing import OneHotEncoder
onehot_enc = OneHotEncoder()
train_cat_enc = onehot_enc.fit_transform(train_cat)
test_cat_enc = onehot_enc.transform(test_cat)
print(train_cat.head(5))
print(f'\nAfter encoding (sparse matrix): \n{train_cat_enc[:5]}')

      ocean_proximity
4781        <1H OCEAN
1512         NEAR BAY
14137      NEAR OCEAN
2912           INLAND
2204           INLAND

After encoding (sparse matrix): 
  (0, 0)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (3, 1)	1.0
  (4, 1)	1.0


Notice that, in order to save memory, this is a SciPy sparse matrix. It can be easily converted to a dense NumPy array by using the toarray() method.

In [22]:
train_cat_enc.toarray()[:5]

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

## Custom transformations
Custom transformations are especially useful for data cleaning and for combining features. For instance, we may want to computer the number of bedrooms per capita or per household, as it is likely to be much more significant than the overall value.

In order to do this, we need to create a new class that implements the fit(), transform() and fit_transform() methods. The latter one can be obtained automatically by subclassing the TransformerMixin class.

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, beds_ix, pop_ix, hh_ix = 3,4,5,6

class CombineAttributes(BaseEstimator, TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room = True, X_is_pd_df = False):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.X_is_pd_df = X_is_pd_df
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.X_is_pd_df: # get and return a pandas DF
            X_new = X
            X_new['rooms_per_household'] = X['total_rooms'] / X['households']
            X_new['avg_household_size'] = X['population'] / X['households']
            if self.add_bedrooms_per_room:
                X_new['bedrooms_per_room'] = X['total_bedrooms'] / X['total_rooms']
            return X_new
        
        else: # get and return a numpy array
            rooms_per_hh = X[:, rooms_ix] / X[:, hh_ix]
            avg_hh_size = X[:, pop_ix] / X[:, hh_ix]
            if self.add_bedrooms_per_room:
                beds_per_room = X[:, beds_ix] / X[:, rooms_ix]
                return np.c_[X, rooms_per_hh, avg_hh_size, beds_per_room]
            else:
                return np.c_[X, rooms_per_hh, avg_hh_size]

In [24]:
comb_attr = CombineAttributes(add_bedrooms_per_room=False, X_is_pd_df=True)
train_num_cstm = comb_attr.transform(train_num)
test_num_cstm = comb_attr.transform(test_num)
train_num_cstm.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,avg_household_size
4781,-118.32,34.04,44.0,1008.0,223.0,544.0,223.0,2.8654,176800.0,4.520179,2.439462
1512,-122.03,37.91,29.0,5438.0,871.0,2310.0,890.0,5.0362,275300.0,6.110112,2.595506
14137,-117.05,32.74,34.0,2178.0,455.0,1193.0,446.0,3.1719,115300.0,4.883408,2.674888
2912,-119.05,35.36,16.0,4507.0,1049.0,2261.0,959.0,3.3261,118400.0,4.699687,2.357664
2204,-119.85,36.82,15.0,1387.0,236.0,638.0,195.0,5.5842,88900.0,7.112821,3.271795


### Scaling numerical features
**Min-max scaling** (often called normalization) shifts and rescales values so that they they up ranging from 0 to 1 (or another provided range). This guarantess maximum stability, however, it is negatively affected by the presence of outliers.

In [25]:
from sklearn.preprocessing import MinMaxScaler
norm_scl = MinMaxScaler()
train_num_norm = norm_scl.fit_transform(train_num_cstm)
test_num_norm = norm_scl.transform(test_num_cstm)
train_num_norm[:,5]

array([0.01894059, 0.08076883, 0.04166229, ..., 0.06525925, 0.04043693,
       0.02590764])

**Standardization** shifts and rescales the data by subtracting its mean and dividing by its standard deviation, so that the resulting distribution has zero mean and unitary variance. Because standardized values are potentially unbounded, it is less afftected by outliers.

In [26]:
from sklearn.preprocessing import StandardScaler
std_scl = StandardScaler()
train_num_std = std_scl.fit_transform(train_num_cstm)
test_num_std = std_scl.transform(test_num_cstm)
train_num_std[:,5]

array([-0.79577658,  0.80422823, -0.20777934, ...,  0.40286802,
       -0.23948951, -0.61548158])

### Transformation Pipelines
All these preceeding steps can be grouped together using a pipeline.

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# pipeline for numerical columns
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('cstm_attribs', CombineAttributes()),
    ('std_scaler', StandardScaler()),
])

train_num_trs = num_pipeline.fit_transform(train_num)
test_num_trs = num_pipeline.transform(test_num)

In [28]:
# pipeline for categorical columns
cat_pipeline = Pipeline([
    ('onehot_enc', OneHotEncoder()),
])

train_cat_trs = cat_pipeline.fit_transform(train_cat)
test_cat_trs = cat_pipeline.transform(test_cat)

In [29]:
# full pipeline
from sklearn.compose import ColumnTransformer

train_cat = train_clean.select_dtypes(exclude=np.number)
train_num = train_clean.select_dtypes(include=np.number)
test_cat = test_clean.select_dtypes(exclude=np.number)
test_num = test_clean.select_dtypes(include=np.number)

num_columns = list(train_num.columns)
cat_columns = list(train_cat.columns)

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_columns),
    ('cat', cat_pipeline, cat_columns),
])

train_prepared = full_pipeline.fit_transform(train_clean)
test_prepared = full_pipeline.transform(test_clean)

In [30]:
train_prepared[:5]

array([[ 0.62065428, -0.74474072,  1.22386067, -0.75003355, -0.75231554,
        -0.79577658, -0.72493135, -0.52648186, -0.25321123, -0.35676013,
        -0.09283641,  0.11433985,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-1.23173899,  1.06568   ,  0.03068215,  1.29781199,  0.80576131,
         0.80422823,  1.0319955 ,  0.62398626,  0.60317048,  0.26105877,
        -0.06757662, -0.81767437,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 1.25476194, -1.35289239,  0.42840832, -0.20918044, -0.19448555,
        -0.20777934, -0.13753302, -0.36404475, -0.78790641, -0.21561609,
        -0.05472651, -0.07375281,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ],
       [ 0.25616719, -0.12723288, -1.00340591,  0.86744084,  1.23375156,
         0.75983399,  1.21374655, -0.28232272, -0.7609543 , -0.28700668,
        -0.10607752,  0.29016044,  0.        ,  1.        ,  0.        ,
         0.        